In [1]:
import nltk
from nltk.tokenize import word_tokenize, WordPunctTokenizer, TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag

!pip install konlpy
import konlpy
from konlpy.tag import Okt, Kkma, Mecab

import re


print(f'\n\n>>> nltk : {nltk.__version__}  |  konlpy : {konlpy.__version__}')

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 40.2MB/s 
     |████████████████████████████████| 92kB 7.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


>>> nltk : 3.2.5  |  konlpy : 0.5.2


## 한글 Bag of words
- Bag of Words : 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
- BoW를 만드는 과정<br>
  1) 각 단어에 고유한 정수 인덱스 부여
  2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터 생성

In [2]:
sent1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
sent2 = "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다."

# 전처리 - 정규 표현식을 통해 온점 제거
sent1 = re.sub("(\.)", "", sent1)
sent2 = re.sub("(\.)", "", sent2)

# OKT 형태소 분석기를 통해 토큰화 수행
okt = Okt()
tokens1 = okt.morphs(sent1)
tokens2 = okt.morphs(sent2)

# 토큰별 index 생성 및 빈도 계산
def get_bow(tokens):
  word2index = {}
  bow = []
  for tk in tokens:
          if tk not in word2index.keys():
              word2index[tk] = len(word2index)
              bow.insert(len(word2index)-1,1)
          else:
              index = word2index.get(tk)
              bow[index]=bow[index]+1
  return word2index, bow


for tmp_tokens in [tokens1, tokens2]:
  tmp_dict, tmp_list = get_bow(tmp_tokens)
  print('word2index :', tmp_dict)
  print('bow :', tmp_list)
  print()

word2index : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bow : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

word2index : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bow : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]

